In [ ]:
!pip install numpy
!pip install d2l --no-deps

In [ ]:
import time
import torch
from torch import nn
from d2l import torch as d2l
from train_helpers import train_and_eval

In [ ]:
accuracies = {}
training_times = {}
parameter_count = {}